<h1>Weibo Post Crawling and Sentiment Analysis</h1>

1. Collect first 50 pages of posts mentioning keywords 俄罗斯 （"Russia" in Chinese) on Weibo between 6:00pm-11:59pm, 6 Mar, 2023. Link of first searching result page: https://s.weibo.com/weibo/%25E8%25B0%2583%25E6%259F%25A5%25E8%25AE%25B0%25E8%2580%2585?q=%E4%BF%84%E7%BD%97%E6%96%AF&typeall=1&suball=1&timescope=custom:2023-03-06-18:2023-03-06-23&Refer=g

The collected data are saved as csv file. Preview: https://guanyipei.github.io/asset/portfolio/web_crawling/russia_posts_on_weibo/Weibo_Crawling_Record.csv

2. Collect information: user_id, user_screen_name, created_at, pid, text, comments, shares, likes

3. Clean and tokenize the text of posts, and detect emotions underlying in the posts by frequency using the emotions dictionary document: https://guanyipei.github.io/asset/portfolio/web_crawling/russia_posts_on_weibo/Chinese_emotion_dictionary.csv

4. Store the deccteed data and do analysis.

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
import pandas as pd
import time
import csv
import jieba
import re

In [2]:
options = webdriver.ChromeOptions() 
options.add_argument("--disable-blink-features=AutomationControlled") 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
options.add_experimental_option("useAutomationExtension", False) 
driver = webdriver.Chrome(options=options) 
driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 

In [3]:
driver.get('https://s.weibo.com/weibo/%25E8%25B0%2583%25E6%259F%25A5%25E8%25AE%25B0%25E8%2580%2585?q=%E4%BF%84%E7%BD%97%E6%96%AF&typeall=1&suball=1&timescope=custom:2023-03-06-18:2023-03-06-23&Refer=g')

In [5]:
# ------------------------
# --Test post collection--
# ------------------------
post_item_list = driver.find_elements(By.XPATH,'//div[@action-type="feed_list_item"]')
print('Length for posts in one page of searching result:',len(post_item_list))
# -----------------------
# ---Test of all items---
# -----------------------
print('--------------')
# user id
print('user id:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="info"]/div[2]/a[@class="name"]').get_attribute('href').split('/')[-1].split('?')[0])
print('--------------')

# user screen name
print('user screen name:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="info"]/div[2]/a[@class="name"]').text)
print('--------------')

# created at (date and time)
print('created at (date and time):',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="from"]/a').text)
print('--------------')

# mid
print('mid:',post_item_list[0].get_attribute('mid'))
print('--------------')

# text
print('text:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/p[@node-type="feed_list_content"]').text)
print('--------------')

#number of sharing
print('Sharing:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[1]/a').text)
# if no sharing activity for one post, it will shows "转发"
print('--------------')

# number of comments
print('Comments:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[2]/a').text)
# if no comment activity for one post, it will shows "评论"
print('--------------')

#number of likes
print('Likes:',post_item_list[0].find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[3]/a').text)
# if no like activity for one post, it will shows "赞"
print('--------------')

Length for posts in one page of searching result: 10
--------------
user id: 7805625463
--------------
user screen name: 津包饺
--------------
created at (date and time): 03月06日 22:59
--------------
mid: 4876429206487187
--------------
text: 各國應轟炸俄羅斯境內“暈輸線”……當年“炮擊金門”很久，最後因美國切斷了“廈門車站”運輸線，炮擊金門才止。（而不應去烏克蘭建軍工廠：俄會集中火力轟炸。）@美国驻华大使馆 @英國駐華使館 @歐盟在中國 @烏克蘭信使
--------------
Sharing: 转发
--------------
Comments: 5
--------------
Likes: 3
--------------


In [7]:
#-----------------------------
#--Test the next page button--
#-----------------------------
next_page_button = driver.find_element(By.XPATH,'//div[@class="m-page"]/div/a[@class="next"]')
next_page_button.click()

In [9]:
file=open('Weibo_Crawling_Record.csv','w',encoding='utf-8',newline='\n')
writer=csv.writer(file)

driver.get('https://s.weibo.com/weibo/%25E8%25B0%2583%25E6%259F%25A5%25E8%25AE%25B0%25E8%2580%2585?q=%E4%BF%84%E7%BD%97%E6%96%AF&typeall=1&suball=1&timescope=custom:2023-03-06-18:2023-03-06-23&Refer=g')
time.sleep(5)

for i in range(50):
    post_item_list = driver.find_elements(By.XPATH,'//div[@action-type="feed_list_item"]')
    for per_post in post_item_list:
        user_id = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="info"]/div[2]/a[@class="name"]').get_attribute('suda-data').split('|')[0].split(':')[-1]
        user_name = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="info"]/div[2]/a[@class="name"]').text
        created_at = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/div[@class="from"]/a').text
        mid = per_post.get_attribute('mid')
        text = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-feed"]/div[@class="content"]/p[@node-type="feed_list_content"]').text
        sharing = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[1]/a').text
        if sharing == '转发':
            sharing = 0
        comments = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[2]/a').text
        if comments == '评论':
            comments = 0
        likes = per_post.find_element(By.XPATH,'./div[@class="card"]/div[@class="card-act"]/ul/li[3]/a').text
        if likes == '赞':
            likes = 0
        writer.writerow([user_id,user_name,created_at,mid,text,sharing,comments,likes])
    print('The info on the page',str(i+1),'has been collected and recorded.')
    next_page_button = driver.find_element(By.XPATH,'//div[@class="m-page"]/div/a[@class="next"]')
    next_page_button.click()
    time.sleep(10)
print('All 50 pages are recorded')
file.close()
print('Done')

The info on the page 1 has been collected and recorded.
The info on the page 2 has been collected and recorded.
The info on the page 3 has been collected and recorded.
The info on the page 4 has been collected and recorded.
The info on the page 5 has been collected and recorded.
The info on the page 6 has been collected and recorded.
The info on the page 7 has been collected and recorded.
The info on the page 8 has been collected and recorded.
The info on the page 9 has been collected and recorded.
The info on the page 10 has been collected and recorded.
The info on the page 11 has been collected and recorded.
The info on the page 12 has been collected and recorded.
The info on the page 13 has been collected and recorded.
The info on the page 14 has been collected and recorded.
The info on the page 15 has been collected and recorded.
The info on the page 16 has been collected and recorded.
The info on the page 17 has been collected and recorded.
The info on the page 18 has been collect

In [10]:
posts_df = pd.read_csv('Weibo_Crawling_Record.csv',header=None)
posts_df.columns = ['User ID','User Screen Name','Created at','mid','Text','Sharing','Comments','Likes']
posts_df = posts_df.drop_duplicates()
posts_df.head()

,User ID,User Screen Name,Created at,mid,Text,Sharing,Comments,Likes
0,168026879741003267738,津包饺,03月06日 22:59,4876429206487187,各國應轟炸俄羅斯境內“暈輸線”……當年“炮擊金門”很久，最後因美國切斷了“廈門車站”運輸線，...,0,5,3
1,168026879741003267738,JRPG斐哥,03月06日 22:58,4876428988124266,这张照片是1992年对岸的RPG爱好者们开办的座谈会。这样的“史前神仙会议”，实在是让我心驰...,25,8,70
2,168026879741003267738,俄罗斯卫星通讯社,03月06日 22:58,4876428979471750,【美国驻安卡拉大使就马克•米利访问叙利亚一事被召唤至土耳其外交部】安卡拉外交消息人士对俄罗斯...,3,3,21
3,168026879741003267738,军事在前沿,03月06日 22:58,4876428955353192,《插手台海挑衅？韩国3架军机直闯台湾空域，中方24架军机紧急起飞》美国等这样的机会已经很久了...,0,0,0
4,168026879741003267738,小猫荐剧,03月06日 22:58,4876428955093819,俄罗斯公布长期盟友名单，中国不在其中，普京良苦用心我们意会了 L小猫荐剧的微博视频,0,0,0


In [11]:
emotion_df=pd.read_csv('Chinese_emotion_dictionary.csv')
emotion_df.head()

,word,sentiment,intensity,valence
0,脏乱,Disgust,7.0,-1.0
1,糟报,Disgust,5.0,-1.0
2,早衰,Disgust,5.0,-1.0
3,责备,Disgust,5.0,-1.0
4,贼眼,Disgust,5.0,-1.0


In [12]:
emotion_dic={}
sentiment_list = emotion_df['sentiment'].value_counts().index.tolist()
for sentiment in sentiment_list:
    emotion_dic[sentiment] = emotion_df[emotion_df['sentiment']==sentiment]['word'].tolist()

In [13]:
def data_cleaning(text):
    text=text.lower()
    text=re.sub('[0-9]+','',text)
    text=re.sub('@[^ ]+','',text)
    text=re.sub('#[^ ]+','',text)
    text=re.sub('https:[^ ]+','',text)
    text=re.sub('[\W]+',' ',text)
    text=text.strip()
    return(text)

In [15]:
result_holder = {} 
for emotion in emotion_dic.keys():
    result_for_one_emotion = []
    for post_text in posts_df['Text'].tolist():
        post_cleaned = data_cleaning(post_text)
        post_splited = list(jieba.cut(post_cleaned))
        count = 0
        for word in emotion_dic[emotion]:
            count += post_splited.count(word)
        result_for_one_emotion.append(count)
    result_holder[emotion] = result_for_one_emotion
for emotion in result_holder.keys():
    posts_df[emotion] = result_holder[emotion]
print(posts_df.info())

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\PEIGUA~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.454 seconds.
Prefix dict has been built successfully.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 476 entries, 0 to 475
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   User ID           476 non-null    object
 1   User Screen Name  476 non-null    object
 2   Created at        476 non-null    object
 3   mid               476 non-null    int64 
 4   Text              476 non-null    object
 5   Sharing           476 non-null    int64 
 6   Comments          476 non-null    int64 
 7   Likes             476 non-null    int64 
 8   Awe               476 non-null    int64 
 9   Disgust           476 non-null    int64 
 10  Sadness           476 non-null    int64 
 11  Joy               476 non-null    int64 
 12  Fear              476 non-null    int64 
 13  Angry             476 non-null    int64 
 14  Surprise          476 non-null    int64 
dtypes: int64(11), object(4)
memory usage: 59.5+ KB
None


In [16]:
posts_df.head()

,User ID,User Screen Name,Created at,mid,Text,Sharing,Comments,Likes,Awe,Disgust,Sadness,Joy,Fear,Angry,Surprise
0,168026879741003267738,津包饺,03月06日 22:59,4876429206487187,各國應轟炸俄羅斯境內“暈輸線”……當年“炮擊金門”很久，最後因美國切斷了“廈門車站”運輸線，...,0,5,3,0,0,0,0,0,0,0
1,168026879741003267738,JRPG斐哥,03月06日 22:58,4876428988124266,这张照片是1992年对岸的RPG爱好者们开办的座谈会。这样的“史前神仙会议”，实在是让我心驰...,25,8,70,7,0,0,0,0,0,0
2,168026879741003267738,俄罗斯卫星通讯社,03月06日 22:58,4876428979471750,【美国驻安卡拉大使就马克•米利访问叙利亚一事被召唤至土耳其外交部】安卡拉外交消息人士对俄罗斯...,3,3,21,0,0,0,0,0,0,0
3,168026879741003267738,军事在前沿,03月06日 22:58,4876428955353192,《插手台海挑衅？韩国3架军机直闯台湾空域，中方24架军机紧急起飞》美国等这样的机会已经很久了...,0,0,0,2,1,0,0,1,0,0
4,168026879741003267738,小猫荐剧,03月06日 22:58,4876428955093819,俄罗斯公布长期盟友名单，中国不在其中，普京良苦用心我们意会了 L小猫荐剧的微博视频,0,0,0,0,0,0,0,0,0,0


How many posts convey angry emotion?

In [17]:
print(len(posts_df[posts_df['Angry']!=0]))

8


Which emotion is most prevalent in this dataset? In other words, which emotion has been mentioned by the most posts?

In [18]:
# this is not the direct answer
result_holder_for_3_2 = {}
for emotion in posts_df.columns[-7:].tolist():
    result_holder_for_3_2[emotion]=posts_df[emotion].sum()
result_holder_for_3_2

{'Awe': 526,
 'Disgust': 292,
 'Sadness': 81,
 'Joy': 129,
 'Fear': 41,
 'Angry': 8,
 'Surprise': 28}

In [19]:
# this is the direct answer
print(max(result_holder_for_3_2, key=result_holder_for_3_2.get))

Awe


An angry post can attract an average of ____ likes. (i.e. How many likes do an angry post obtain on average)

In [21]:
angry_post = posts_df[posts_df['Angry']!=0]
print(angry_post['Likes'].mean())

18.625


Find if there is any relationship between emotions and interactions (comments, sharing, likes).

In [22]:
import plotly.graph_objs as go

In [23]:
posts_df['Emotion']=posts_df.iloc[:,-7:-1].sum(axis=1)
posts_df['Interaction']=posts_df.iloc[:,-11:-9].sum(axis=1)

posts_without_emotion = posts_df[posts_df['Emotion']==0]
posts_with_emotion = posts_df[posts_df['Emotion']!=0]
posts_with_awe =  posts_df[posts_df['Awe']!=0]
posts_with_dis =  posts_df[posts_df['Disgust']!=0]
posts_with_sad =  posts_df[posts_df['Sadness']!=0]
posts_with_joy =  posts_df[posts_df['Joy']!=0]
posts_with_fea =  posts_df[posts_df['Fear']!=0]
posts_with_ang =  posts_df[posts_df['Angry']!=0]
posts_with_sur =  posts_df[posts_df['Surprise']!=0]

category_list = [
    'Post without Emotion',
    'Post with Emotion',
    'Awe Prevailed Posts',
    'Disgust Prevailed Posts',
    'Sad Prevailed Posts',
    'Joy Prevailed Posts',
    'Fear Prevailed Posts',
    'Angry Prevailed Posts',
    'Surprise Prevailed Posts',
]
interaction_score =  [
    posts_without_emotion['Interaction'].mean(),
    posts_with_emotion['Interaction'].mean(),
    posts_with_awe['Interaction'].mean(),
    posts_with_dis['Interaction'].mean(),
    posts_with_sad['Interaction'].mean(),
    posts_with_joy['Interaction'].mean(),
    posts_with_fea['Interaction'].mean(),
    posts_with_ang['Interaction'].mean(),
    posts_with_sur['Interaction'].mean(),
]

trace = go.Bar(x=category_list,y=interaction_score,name='Average Interaction Received',marker={'color':['red','red','blue','blue','blue','blue','blue','blue','blue','blue']})
layout = go.Layout(title='The average received interactions for posts with different emotion presents')
figure = go.Figure(data=trace,layout=layout)
figure.show()

In the searching results of keywards Russia during a period Russia-Ukraine War is still undertaking,<br>
posts that do not contain any emotion presentation evaluated by keywords frequency got far less (routhly half) interactions than those with emotions by audience on average,<br>
and within the posts containing emotions, different emotions may have different accration in terms of catching interaction.<br>
Disgust, awe, surprise, and sad emotions may attract more interactions to some degree,<br>
which may be a reference for politicians or other public speakers who want to draw public attentions or even mobilize public sentiment.
